<a href="https://colab.research.google.com/github/MikeyBoo/GoogleColab/blob/master/My_nMigen_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing SymbiYosys and Other Dependencies
#### various notes
Gaining root previlges on Ubuntu systems, use hte follwing commands while opening Jupyter Notebook:

sudo bash

jupyter notebook --allow-root

#### Building on Raspberry Pi
To avoid thrashing on Raspberry Pi

!export MAX_JOBS=2

In [0]:
!sudo apt-get update --yes
!sudo apt-get upgrade --yes
!sudo apt-get install --yes curl
!sudo apt-get install -y --fix-missing build-essential clang bison flex libreadline-dev \
                     gawk tcl-dev libssl-dev libffi-dev git mercurial graphviz   \
                     xdot pkg-config python python3 python-dev libftdi-dev gperf \
                     libboost-program-options-dev autoconf libgmp-dev cmake

In [0]:
%cd ~
!mkdir nMigen_Project
%cd nMigen_Project

In [0]:
!git clone https://github.com/YosysHQ/yosys.git yosys
%cd yosys
!make -j$(nproc)
!sudo make install
%cd ~/nMigen_Project

In [0]:
!git clone https://github.com/YosysHQ/SymbiYosys.git SymbiYosys
%cd SymbiYosys
!sudo make install
%cd ~/nMigen_Project

In [0]:
!git clone https://github.com/SRI-CSL/yices2.git yices2
%cd yices2
!autoconf
!./configure
!make -j$(nproc)
!sudo make install
%cd ~/nMigen_Project

In [0]:
!git clone https://github.com/Z3Prover/z3.git z3
%cd z3
!python scripts/mk_make.py
%cd build
!make -j$(nproc)
!sudo make install
%cd ~/nMigen_Project

In [0]:
!git clone https://github.com/boolector/boolector
%cd boolector
!./contrib/setup-btor2tools.sh
!./contrib/setup-lingeling.sh
!./configure.sh
!make -C build -j$(nproc)
!sudo cp build/bin/{boolector,btor*} /usr/local/bin/
!sudo cp deps/btor2tools/bin/btorsim /usr/local/bin/
%cd ~/nMigen_Project

## Collecting Required Python Modules
!{sys.executable} runs proper python environment used within current Jupyter Notebook

In [0]:
import sys
!{sys.executable} -m pip install wheel
!{sys.executable} -m pip install git+https://github.com/m-labs/nmigen.git

In [0]:
!sudo apt-get install --yes gtkwave

In [0]:
!sudo apt-get install --yes --fix-missing build-essential clang bison flex libreadline-dev \
                     gawk tcl-dev libffi-dev git mercurial graphviz   \
                     xdot pkg-config python python3 libftdi-dev \
                     qt5-default python3-dev libboost-all-dev cmake libeigen3-dev

In [0]:
!sudo apt-get install --yes g++-7 gcc-7

In [0]:
!sudo apt-get install --yes qt5-base qt5-default

In [0]:
!git clone https://github.com/cliffordwolf/icestorm.git icestorm
%cd icestorm
!make -j$(nproc)
!sudo make install
%cd ~/nMigen_Project

In [0]:
!sudo strip --remove-section=.note.ABI-tag /usr/lib/x86_64-linux-gnu/libQt5Core.so.5

In [0]:
!git clone https://github.com/YosysHQ/nextpnr nextpnr
%cd nextpnr
!cmake -DARCH=ice40 -DCMAKE_INSTALL_PREFIX=/usr/local .
!make -j$(nproc)
!sudo make install
%cd ~/nMigen_Project

In [0]:
from nmigen import *

# Shapes

In [0]:
a = Const(10)

In [0]:
a.shape()

In [0]:
a.width

In [0]:
a.signed

In [0]:
a = Const(-10)

In [0]:
a.shape()

In [0]:
x = Const(3, range(-5,11))
x.shape()

In [0]:
from enum import Enum, unique

finds minimum and maximum value of the enum and then using that as the range for the constant

In [0]:
@unique
class Func(Enum):
    NONE = 0
    ADD = 1
    SUB = 2
    MUL = 3
    DIV = 4

In [0]:
x = Const(2, Func)
x.shape()

# Signals

In [0]:
a = Signal(signed(16))
a.shape()


In [0]:
a.width

In [0]:
a.signed

#### Signal from range

In [0]:
Signal(range(11))

In [0]:
x = Signal(range(-5, 11))

In [0]:
x.shape()

In [0]:
@unique
class Func(Enum):
    NONE = 0
    ADD = 1
    SUB = 2
    MUL = 3
    DIV = 4
    
x = Signal(Func)

##Same is Signal.range(0, 5)

In [0]:
x.shape()

In [0]:
x = Signal(unsigned(16))
x.name

In [0]:
x = Signal(unsigned(16), name="addr")
x.name

In [0]:
!gtkwave

## Modules
#### Basic structure

In [0]:
from nmigen.build import Platform

class ThingBlock(Elaboratable):
    def __init__(self):
        pass
    
    def elaborate(self, platform: Platform) -> Module:
        m = Module()
        return m

#### Elaborating a module

In [0]:
%%writefile thing.py

from nmigen import *
from nmigen.build import Platform

class ThingBlock(Elaboratable):
    def __init__(self):
        pass

    def elaborate(self, platform: Platform) -> Module:
        m = Module()
        return m

from nmigen.cli import main

if __name__ == "__main__":
    sync = ClockDomain()
    
    block = ThingBlock()
    
    m = Module()
    m.domains += sync
    m.submodules += block
    
    main(m, ports=[sync.clk, sync.rst])
    

## Domains

A domain, in its basic definition, is a grouping of logic elements. If we consider a module as a black box with inputs and outputs, then any given output is generated within one and only one domain. If you attempt to set an output in more than one domain, you'll get an error during elaboration that the signal has more than one driver -- a "driver-driver conflict".

Modules come with two domains built in: a combinatorial domain and a synchronous domain.

The domains in a Module can be accessed through its d attribute.

In [0]:
!{sys.executable} thing.py generate -t v > thing.v

In [0]:
!ls

In [0]:
m.d.comb


In [0]:
m.d.sync

In [0]:
m.d.myclk

In [0]:
m = Module()
c = ClockDomain(myclk)

m.domains += c

## Ports
The equivalent of ports in a module is public attributes. In the following example, a and data are publically available to other modules, while b is not, just as a and data are publically available to other Python classes, and b is not.

In [0]:
class ThingBlock(Elaboratable):
    def __init__(self):
        self.a = Signal()
        self.data = Signal(8)
        
    def elaborate(self, platofrm: str):
        m = Module()
        
        b = Signal()
        
        return m

If a signal is set in the combinatorial domain, then you can specify the default value of the signal if it is not set. By default, this is zero, but for a non-zero value, you can specify the default value for a signal when constructing the signal by setting the reset named parameter in the constructor. For example, this creates a 16-bit unsigned signal, self.x, which defaults to 0x1000 if not set:

In [0]:
x = Signal(unsigned(16), reset=0x1000)

In [0]:
x.shape()

This would create a 16-bit unsigned signal that is initially set to 0x1000, but is not reset to that value when the domain's reset signal is activated.

In [0]:
x = Signal(unsigned(16), reset=0x1000, reset_less=True)

In [0]:
s1 = Signal(4)
s2 = Signal(4)
v = s1 + s2
v.shape()

## Multiplexing signals

In [0]:
y = Signal(4)
cond = True
y.eq(Mux(cond, s1, s2))

## Placing statements in domains

Statements are written in the combinatorial domain of a module, or in a sequential domain (clock domain) of a module. The equivalent in Verilog is continuous assignment and clocked assignment.

So if you have a module m, you can add a statement that x gets the value of y+1 all the time like this:

m.d.comb += x.eq(y+1)

On the other hand, if you have a clock domain sync that is clocked on the positive edge, then you can add a statement that x gets the value of y+1 on the positive edge of the clock of sync like this:

m.d.sync += x.eq(y+1)

The += operator for a domain can take one statement, or an array of statements, which can be convenient:

m.d.comb += [
    x.eq(y+1),
    z.eq(w+2),
]

In [0]:
!nextpnr-ice40 --gui

In [0]:
!nextpnr